## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import joblib
tqdm.pandas()

## Data Load

In [ ]:
BASE_DIR = "./data"

train_meta = pd.read_csv(os.path.join(BASE_DIR, "/content/drive/MyDrive/data/train.csv"))
train_A = pd.read_csv(os.path.join(BASE_DIR, "/content/drive/MyDrive/data/train/A.csv"))
train_B = pd.read_csv(os.path.join(BASE_DIR, "/content/drive/MyDrive/data/train/B.csv"))

print("train_meta:", train_meta.shape)
print("train_A:", train_A.shape)
print("train_B:", train_B.shape)

train_meta: (944767, 3)
train_A: (647241, 37)
train_B: (297526, 31)


## Data Pre-processing

In [ ]:
def convert_age(val):
    if pd.isna(val): return np.nan
    try:
        base = int(str(val)[:-1])
        return base if str(val)[-1] == "a" else base + 5
    except:
        return np.nan

def split_testdate(val):
    try:
        v = int(val)
        return v // 100, v % 100
    except:
        return np.nan, np.nan

def seq_mean(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").mean() if x else np.nan
    )

def seq_std(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").std() if x else np.nan
    )

def seq_rate(series, target="1"):
    return series.fillna("").progress_apply(
        lambda x: str(x).split(",").count(target) / len(x.split(",")) if x else np.nan
    )

def masked_mean_from_csv_series(cond_series, val_series, mask_val):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)

    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)

    mask = (cond_arr == mask_val)
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts==0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)

def masked_mean_in_set_series(cond_series, val_series, mask_set):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)

    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)

    mask = np.isin(cond_arr, list(mask_set))
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts == 0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)

## 1차 Feature Engineering

In [ ]:
def preprocess_A(train_A):
    df = train_A.copy()

    # ---- Age, TestDate 파생 ----
    print("Step 1: Age, TestDate 파생...")
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)

    # ---- A1 ----
    print("Step 2: A1 feature 생성...")
    feats["A1_resp_rate"] = seq_rate(df["A1-3"], "1")
    feats["A1_rt_mean"]   = seq_mean(df["A1-4"])
    feats["A1_rt_std"]    = seq_std(df["A1-4"])
    feats["A1_rt_left"]   = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 1)
    feats["A1_rt_right"]  = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 2)
    feats["A1_rt_side_diff"] = feats["A1_rt_left"] - feats["A1_rt_right"]
    feats["A1_rt_slow"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 1)
    feats["A1_rt_fast"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 3)
    feats["A1_rt_speed_diff"] = feats["A1_rt_slow"] - feats["A1_rt_fast"]

    # ---- A2 ----
    print("Step 3: A2 feature 생성...")
    feats["A2_resp_rate"] = seq_rate(df["A2-3"], "1")
    feats["A2_rt_mean"]   = seq_mean(df["A2-4"])
    feats["A2_rt_std"]    = seq_std(df["A2-4"])
    feats["A2_rt_cond1_diff"] = masked_mean_from_csv_series(df["A2-1"], df["A2-4"], 1) - \
                                masked_mean_from_csv_series(df["A2-1"], df["A2-4"], 3)
    feats["A2_rt_cond2_diff"] = masked_mean_from_csv_series(df["A2-2"], df["A2-4"], 1) - \
                                masked_mean_from_csv_series(df["A2-2"], df["A2-4"], 3)

    # ---- A3 ----
    print("Step 4: A3 feature 생성...")
    s = df["A3-5"].fillna("")
    total   = s.apply(lambda x: len(x.split(",")) if x else 0)
    valid   = s.apply(lambda x: sum(v in {"1","2"} for v in x.split(",")) if x else 0)
    invalid = s.apply(lambda x: sum(v in {"3","4"} for v in x.split(",")) if x else 0)
    correct = s.apply(lambda x: sum(v in {"1","3"} for v in x.split(",")) if x else 0)
    feats["A3_valid_ratio"]   = (valid / total).replace([np.inf,-np.inf], np.nan)
    feats["A3_invalid_ratio"] = (invalid / total).replace([np.inf,-np.inf], np.nan)
    feats["A3_correct_ratio"] = (correct / total).replace([np.inf,-np.inf], np.nan)

    feats["A3_resp2_rate"] = seq_rate(df["A3-6"], "1")
    feats["A3_rt_mean"]    = seq_mean(df["A3-7"])
    feats["A3_rt_std"]     = seq_std(df["A3-7"])
    feats["A3_rt_size_diff"] = masked_mean_from_csv_series(df["A3-1"], df["A3-7"], 1) - \
                               masked_mean_from_csv_series(df["A3-1"], df["A3-7"], 2)
    feats["A3_rt_side_diff"] = masked_mean_from_csv_series(df["A3-3"], df["A3-7"], 1) - \
                               masked_mean_from_csv_series(df["A3-3"], df["A3-7"], 2)

    # ---- A4 ----
    print("Step 5: A4 feature 생성...")
    feats["A4_acc_rate"]   = seq_rate(df["A4-3"], "1")
    feats["A4_resp2_rate"] = seq_rate(df["A4-4"], "1")
    feats["A4_rt_mean"]    = seq_mean(df["A4-5"])
    feats["A4_rt_std"]     = seq_std(df["A4-5"])
    feats["A4_stroop_diff"] = masked_mean_from_csv_series(df["A4-1"], df["A4-5"], 2) - \
                              masked_mean_from_csv_series(df["A4-1"], df["A4-5"], 1)
    feats["A4_rt_color_diff"] = masked_mean_from_csv_series(df["A4-2"], df["A4-5"], 1) - \
                                masked_mean_from_csv_series(df["A4-2"], df["A4-5"], 2)

    # ---- A5 ----
    print("Step 6: A5 feature 생성...")
    feats["A5_acc_rate"]   = seq_rate(df["A5-2"], "1")
    feats["A5_resp2_rate"] = seq_rate(df["A5-3"], "1")
    feats["A5_acc_nonchange"] = masked_mean_from_csv_series(df["A5-1"], df["A5-2"], 1)
    feats["A5_acc_change"]    = masked_mean_in_set_series(df["A5-1"], df["A5-2"], {2,3,4})

    # ---- Drop ----
    print("Step 7: 시퀀스 컬럼 drop & concat...")
    seq_cols = [
        "A1-1","A1-2","A1-3","A1-4",
        "A2-1","A2-2","A2-3","A2-4",
        "A3-1","A3-2","A3-3","A3-4","A3-5","A3-6","A3-7",
        "A4-1","A4-2","A4-3","A4-4","A4-5",
        "A5-1","A5-2","A5-3"
    ]
    print("A 검사 데이터 전처리 완료")
    return pd.concat([df.drop(columns=seq_cols, errors="ignore"), feats], axis=1)

In [ ]:
def preprocess_B(train_B):
    df = train_B.copy()

    # ---- Age, TestDate ----
    print("Step 1: Age, TestDate 파생...")
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)

    # ---- B1 ----
    print("Step 2: B1 feature 생성...")
    feats["B1_acc_task1"] = seq_rate(df["B1-1"], "1")
    feats["B1_rt_mean"]   = seq_mean(df["B1-2"])
    feats["B1_rt_std"]    = seq_std(df["B1-2"])
    feats["B1_acc_task2"] = seq_rate(df["B1-3"], "1")

    # ---- B2 ----
    print("Step 3: B2 feature 생성...")
    feats["B2_acc_task1"] = seq_rate(df["B2-1"], "1")
    feats["B2_rt_mean"]   = seq_mean(df["B2-2"])
    feats["B2_rt_std"]    = seq_std(df["B2-2"])
    feats["B2_acc_task2"] = seq_rate(df["B2-3"], "1")

    # ---- B3 ----
    print("Step 4: B3 feature 생성...")
    feats["B3_acc_rate"] = seq_rate(df["B3-1"], "1")
    feats["B3_rt_mean"]  = seq_mean(df["B3-2"])
    feats["B3_rt_std"]   = seq_std(df["B3-2"])

    # ---- B4 ----
    print("Step 5: B4 feature 생성...")
    feats["B4_acc_rate"] = seq_rate(df["B4-1"], "1")
    feats["B4_rt_mean"]  = seq_mean(df["B4-2"])
    feats["B4_rt_std"]   = seq_std(df["B4-2"])

    # ---- B5 ----
    print("Step 6: B5 feature 생성...")
    feats["B5_acc_rate"] = seq_rate(df["B5-1"], "1")
    feats["B5_rt_mean"]  = seq_mean(df["B5-2"])
    feats["B5_rt_std"]   = seq_std(df["B5-2"])

    # ---- B6~B8 ----
    print("Step 7: B6~B8 feature 생성...")
    feats["B6_acc_rate"] = seq_rate(df["B6"], "1")
    feats["B7_acc_rate"] = seq_rate(df["B7"], "1")
    feats["B8_acc_rate"] = seq_rate(df["B8"], "1")

    # ---- Drop ----
    print("Step 8: 시퀀스 컬럼 drop & concat...")
    seq_cols = [
        "B1-1","B1-2","B1-3",
        "B2-1","B2-2","B2-3",
        "B3-1","B3-2",
        "B4-1","B4-2",
        "B5-1","B5-2",
        "B6","B7","B8"
    ]

    print("B 검사 데이터 전처리 완료")
    return pd.concat([df.drop(columns=seq_cols, errors="ignore"), feats], axis=1)

In [ ]:
train_A_features = preprocess_A(train_A)
train_B_features = preprocess_B(train_B)

print("A:", train_A_features.shape, "B:", train_B_features.shape)

Step 1: Age, TestDate 파생...
Step 2: A1 feature 생성...


100%|██████████| 647241/647241 [00:21<00:00, 30443.08it/s]


Step 3: A2 feature 생성...


100%|██████████| 647241/647241 [00:20<00:00, 31340.88it/s]


Step 4: A3 feature 생성...


...

Step 6: B5 feature 생성...


100%|██████████| 297526/297526 [00:10<00:00, 28204.60it/s]


Step 7: B6~B8 feature 생성...


100%|██████████| 297526/297526 [00:00<00:00, 741649.18it/s]


Step 8: 시퀀스 컬럼 drop & concat...
B 검사 데이터 전처리 완료
A: (647241, 49) B: (297526, 39)


## 2차 Feature Engineering

In [ ]:
# -------- 공통 유틸 --------
def _has(df, cols):  # 필요한 컬럼이 모두 있는지
    return all(c in df.columns for c in cols)

def _safe_div(a, b, eps=1e-6):
    return a / (b + eps)

# -------- A 파생 --------
def add_features_A(df: pd.DataFrame) -> pd.DataFrame:
    feats = df.copy()
    eps = 1e-6

    # 0) Year-Month 단일축
    if _has(feats, ["Year","Month"]):
        feats["YearMonthIndex"] = feats["Year"] * 12 + feats["Month"]

    # 1) 속도-정확도 트레이드오프
    if _has(feats, ["A1_rt_mean","A1_resp_rate"]):
        feats["A1_speed_acc_tradeoff"] = _safe_div(feats["A1_rt_mean"], feats["A1_resp_rate"], eps)
    if _has(feats, ["A2_rt_mean","A2_resp_rate"]):
        feats["A2_speed_acc_tradeoff"] = _safe_div(feats["A2_rt_mean"], feats["A2_resp_rate"], eps)
    if _has(feats, ["A4_rt_mean","A4_acc_rate"]):
        feats["A4_speed_acc_tradeoff"] = _safe_div(feats["A4_rt_mean"], feats["A4_acc_rate"], eps)

    # 2) RT 변동계수(CV)
    for k in ["A1","A2","A3","A4"]:
        m, s = f"{k}_rt_mean", f"{k}_rt_std"
        if _has(feats, [m, s]):
            feats[f"{k}_rt_cv"] = _safe_div(feats[s], feats[m], eps)

    # 3) 조건 차이 절댓값(편향 크기)
    for name, base in [
        ("A1_rt_side_gap_abs",  "A1_rt_side_diff"),
        ("A1_rt_speed_gap_abs", "A1_rt_speed_diff"),
        ("A2_rt_cond1_gap_abs", "A2_rt_cond1_diff"),
        ("A2_rt_cond2_gap_abs", "A2_rt_cond2_diff"),
        ("A4_stroop_gap_abs",   "A4_stroop_diff"),
        ("A4_color_gap_abs",    "A4_rt_color_diff"),
    ]:
        if base in feats.columns:
            feats[name] = feats[base].abs()

    # 4) 정확도 패턴 심화
    if _has(feats, ["A3_valid_ratio","A3_invalid_ratio"]):
        feats["A3_valid_invalid_gap"] = feats["A3_valid_ratio"] - feats["A3_invalid_ratio"]
    if _has(feats, ["A3_correct_ratio","A3_invalid_ratio"]):
        feats["A3_correct_invalid_gap"] = feats["A3_correct_ratio"] - feats["A3_invalid_ratio"]
    if _has(feats, ["A5_acc_change","A5_acc_nonchange"]):
        feats["A5_change_nonchange_gap"] = feats["A5_acc_change"] - feats["A5_acc_nonchange"]

    # 5) 간단 메타 리스크 스코어(휴리스틱)
    parts = []
    if "A4_stroop_gap_abs" in feats: parts.append(0.30 * feats["A4_stroop_gap_abs"].fillna(0))
    if "A4_acc_rate" in feats:       parts.append(0.20 * (1 - feats["A4_acc_rate"].fillna(0)))
    if "A3_valid_invalid_gap" in feats:
        parts.append(0.20 * feats["A3_valid_invalid_gap"].fillna(0).abs())
    if "A1_rt_cv" in feats: parts.append(0.20 * feats["A1_rt_cv"].fillna(0))
    if "A2_rt_cv" in feats: parts.append(0.10 * feats["A2_rt_cv"].fillna(0))
    if parts:
        feats["RiskScore"] = sum(parts)

    # NaN/inf 정리
    feats.replace([np.inf, -np.inf], np.nan, inplace=True)
    return feats

# -------- B 파생 --------
def add_features_B(df: pd.DataFrame) -> pd.DataFrame:
    feats = df.copy()
    eps = 1e-6

    # 0) Year-Month 단일축
    if _has(feats, ["Year","Month"]):
        feats["YearMonthIndex"] = feats["Year"] * 12 + feats["Month"]

    # 1) 속도-정확도 트레이드오프 (B1~B5)
    for k, acc_col, rt_col in [
        ("B1", "B1_acc_task1", "B1_rt_mean"),
        ("B2", "B2_acc_task1", "B2_rt_mean"),
        ("B3", "B3_acc_rate",  "B3_rt_mean"),
        ("B4", "B4_acc_rate",  "B4_rt_mean"),
        ("B5", "B5_acc_rate",  "B5_rt_mean"),
    ]:
        if _has(feats, [rt_col, acc_col]):
            feats[f"{k}_speed_acc_tradeoff"] = _safe_div(feats[rt_col], feats[acc_col], eps)

    # 2) RT 변동계수(CV)
    for k in ["B1","B2","B3","B4","B5"]:
        m, s = f"{k}_rt_mean", f"{k}_rt_std"
        if _has(feats, [m, s]):
            feats[f"{k}_rt_cv"] = _safe_div(feats[s], feats[m], eps)

    # 3) 간단 메타 리스크 스코어(휴리스틱)
    parts = []
    for k in ["B4","B5"]:  # 주의집중/스트룹 유사 과제 가중
        if _has(feats, [f"{k}_rt_cv"]):
            parts.append(0.25 * feats[f"{k}_rt_cv"].fillna(0))
    for k in ["B3","B4","B5"]:
        acc = f"{k}_acc_rate" if k != "B1" and k != "B2" else None
        if k in ["B1","B2"]:
            acc = f"{k}_acc_task1"
        if acc in feats:
            parts.append(0.25 * (1 - feats[acc].fillna(0)))
    for k in ["B1","B2"]:
        tcol = f"{k}_speed_acc_tradeoff"
        if tcol in feats:
            parts.append(0.25 * feats[tcol].fillna(0))
    if parts:
        feats["RiskScore_B"] = sum(parts)

    feats.replace([np.inf, -np.inf], np.nan, inplace=True)
    return feats

In [ ]:
train_A_features = add_features_A(train_A_features)
train_B_features = add_features_B(train_B_features)

print("A+feat:", train_A_features.shape, "B+feat:", train_B_features.shape)

A+feat: (647241, 67) B+feat: (297526, 51)


## Train / Validation Split

In [ ]:
meta_A = train_meta[train_meta["Test"]=="A"].reset_index(drop=True)
meta_B = train_meta[train_meta["Test"]=="B"].reset_index(drop=True)

X_A, y_A = train_A_features.drop(columns=["Test_id","Test","PrimaryKey","Age","TestDate"]), meta_A["Label"].values
X_B, y_B = train_B_features.drop(columns=["Test_id","Test","PrimaryKey","Age","TestDate"]), meta_B["Label"].values

X_train_A, X_val_A, y_train_A, y_val_A = train_test_split(X_A, y_A, test_size=0.2, stratify=y_A, random_state=42)
X_train_B, X_val_B, y_train_B, y_val_B = train_test_split(X_B, y_B, test_size=0.2, stratify=y_B, random_state=42)

## Model Train

In [ ]:
def train_and_eval(X_train, y_train, X_val, y_val, group_label):
    model = lgb.LGBMClassifier(
        objective="binary",
        metric="auc",
        n_estimators=3000,
        learning_rate=0.05,
        n_jobs=-1,
        random_state=42,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]
    )

    val_pred = model.predict_proba(X_val)[:,1]
    auc = roc_auc_score(y_val, val_pred)
    print(f"[{group_label}] Validation AUC: {auc:.4f}")
    return model

In [ ]:
model_A = train_and_eval(X_train_A, y_train_A, X_val_A, y_val_A, "A")
model_B = train_and_eval(X_train_B, y_train_B, X_val_B, y_val_B, "B")

[LightGBM] [Info] Number of positive: 11754, number of negative: 506038
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.534830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9626
[LightGBM] [Info] Number of data points in the train set: 517792, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022700 -> initscore=-3.762418
[LightGBM] [Info] Start training from score -3.762418
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.504672
[200]	valid_0's auc: 0.508239
[300]	valid_0's auc: 0.508657
[400]	valid_0's auc: 0.507633
Early stopping, best iteration is:
[254]	valid_0's auc: 0.510804
[A] Validation AUC: 0.5108
[LightGBM] [Info] Number of positive: 10072, number of negative: 227948
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043428 seconds.
You can set `force_row_wise=true` to remove the overhe

## Save Model

In [ ]:
import os
import joblib

# 모델 저장 경로
os.makedirs("./model", exist_ok=True)

joblib.dump(model_A, "./model/lgbm_A.pkl")
joblib.dump(model_B, "./model/lgbm_B.pkl")

print("모델 저장 완료: ./model/lgbm_A.pkl, ./model/lgbm_B.pkl")

모델 저장 완료: ./model/lgbm_A.pkl, ./model/lgbm_B.pkl


#Model inference

In [ ]:
# 학습 때 사용한 전처리 유틸 (그대로)
tqdm.pandas()

def convert_age(val):
    if pd.isna(val): return np.nan
    try:
        base = int(str(val)[:-1])
        return base if str(val)[-1] == "a" else base + 5
    except:
        return np.nan

def split_testdate(val):
    try:
        v = int(val)
        return v // 100, v % 100
    except:
        return np.nan, np.nan

def seq_mean(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").mean() if x else np.nan
    )

def seq_std(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").std() if x else np.nan
    )

def seq_rate(series, target="1"):
    return series.fillna("").progress_apply(
        lambda x: str(x).split(",").count(target) / len(x.split(",")) if x else np.nan
    )

def masked_mean_from_csv_series(cond_series, val_series, mask_val):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)
    mask = (cond_arr == mask_val)
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts==0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)

def masked_mean_in_set_series(cond_series, val_series, mask_set):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)
    mask = np.isin(cond_arr, list(mask_set))
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts == 0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)


In [ ]:
# 학습 때 사용한 A/B 검사 전처리 (그대로)
def preprocess_A(train_A):
    df = train_A.copy()
    print("Step 1: Age, TestDate 파생...")
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)

    print("Step 2: A1 feature 생성...")
    feats["A1_resp_rate"] = seq_rate(df["A1-3"], "1")
    feats["A1_rt_mean"]   = seq_mean(df["A1-4"])
    feats["A1_rt_std"]    = seq_std(df["A1-4"])
    feats["A1_rt_left"]   = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 1)
    feats["A1_rt_right"]  = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 2)
    feats["A1_rt_side_diff"] = feats["A1_rt_left"] - feats["A1_rt_right"]
    feats["A1_rt_slow"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 1)
    feats["A1_rt_fast"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 3)
    feats["A1_rt_speed_diff"] = feats["A1_rt_slow"] - feats["A1_rt_fast"]

    print("Step 3: A2 feature 생성...")
    feats["A2_resp_rate"] = seq_rate(df["A2-3"], "1")
    feats["A2_rt_mean"]   = seq_mean(df["A2-4"])
    feats["A2_rt_std"]    = seq_std(df["A2-4"])
    feats["A2_rt_cond1_diff"] = masked_mean_from_csv_series(df["A2-1"], df["A2-4"], 1) - \
                                masked_mean_from_csv_series(df["A2-1"], df["A2-4"], 3)
    feats["A2_rt_cond2_diff"] = masked_mean_from_csv_series(df["A2-2"], df["A2-4"], 1) - \
                                masked_mean_from_csv_series(df["A2-2"], df["A2-4"], 3)

    print("Step 4: A3 feature 생성...")
    s = df["A3-5"].fillna("")
    total   = s.apply(lambda x: len(x.split(",")) if x else 0)
    valid   = s.apply(lambda x: sum(v in {"1","2"} for v in x.split(",")) if x else 0)
    invalid = s.apply(lambda x: sum(v in {"3","4"} for v in x.split(",")) if x else 0)
    correct = s.apply(lambda x: sum(v in {"1","3"} for v in x.split(",")) if x else 0)
    feats["A3_valid_ratio"]   = (valid / total).replace([np.inf,-np.inf], np.nan)
    feats["A3_invalid_ratio"] = (invalid / total).replace([np.inf,-np.inf], np.nan)
    feats["A3_correct_ratio"] = (correct / total).replace([np.inf,-np.inf], np.nan)

    feats["A3_resp2_rate"] = seq_rate(df["A3-6"], "1")
    feats["A3_rt_mean"]    = seq_mean(df["A3-7"])
    feats["A3_rt_std"]     = seq_std(df["A3-7"])
    feats["A3_rt_size_diff"] = masked_mean_from_csv_series(df["A3-1"], df["A3-7"], 1) - \
                               masked_mean_from_csv_series(df["A3-1"], df["A3-7"], 2)
    feats["A3_rt_side_diff"] = masked_mean_from_csv_series(df["A3-3"], df["A3-7"], 1) - \
                               masked_mean_from_csv_series(df["A3-3"], df["A3-7"], 2)

    print("Step 5: A4 feature 생성...")
    feats["A4_acc_rate"]   = seq_rate(df["A4-3"], "1")
    feats["A4_resp2_rate"] = seq_rate(df["A4-4"], "1")
    feats["A4_rt_mean"]    = seq_mean(df["A4-5"])
    feats["A4_rt_std"]     = seq_std(df["A4-5"])
    feats["A4_stroop_diff"] = masked_mean_from_csv_series(df["A4-1"], df["A4-5"], 2) - \
                              masked_mean_from_csv_series(df["A4-1"], df["A4-5"], 1)
    feats["A4_rt_color_diff"] = masked_mean_from_csv_series(df["A4-2"], df["A4-5"], 1) - \
                                masked_mean_from_csv_series(df["A4-2"], df["A4-5"], 2)

    print("Step 6: A5 feature 생성...")
    feats["A5_acc_rate"]   = seq_rate(df["A5-2"], "1")
    feats["A5_resp2_rate"] = seq_rate(df["A5-3"], "1")
    feats["A5_acc_nonchange"] = masked_mean_from_csv_series(df["A5-1"], df["A5-2"], 1)
    feats["A5_acc_change"]    = masked_mean_in_set_series(df["A5-1"], df["A5-2"], {2,3,4})

    print("Step 7: 시퀀스 컬럼 drop & concat...")
    seq_cols = [
        "A1-1","A1-2","A1-3","A1-4",
        "A2-1","A2-2","A2-3","A2-4",
        "A3-1","A3-2","A3-3","A3-4","A3-5","A3-6","A3-7",
        "A4-1","A4-2","A4-3","A4-4","A4-5",
        "A5-1","A5-2","A5-3"
    ]
    print("A 검사 데이터 전처리 완료")
    out = pd.concat([df.drop(columns=seq_cols, errors="ignore"), feats], axis=1)
    out.replace([np.inf,-np.inf], np.nan, inplace=True)
    return out

def preprocess_B(train_B):
    df = train_B.copy()
    print("Step 1: Age, TestDate 파생...")
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)

    print("Step 2: B1 feature 생성...")
    feats["B1_acc_task1"] = seq_rate(df["B1-1"], "1")
    feats["B1_rt_mean"]   = seq_mean(df["B1-2"])
    feats["B1_rt_std"]    = seq_std(df["B1-2"])
    feats["B1_acc_task2"] = seq_rate(df["B1-3"], "1")

    print("Step 3: B2 feature 생성...")
    feats["B2_acc_task1"] = seq_rate(df["B2-1"], "1")
    feats["B2_rt_mean"]   = seq_mean(df["B2-2"])
    feats["B2_rt_std"]    = seq_std(df["B2-2"])
    feats["B2_acc_task2"] = seq_rate(df["B2-3"], "1")

    print("Step 4: B3 feature 생성...")
    feats["B3_acc_rate"] = seq_rate(df["B3-1"], "1")
    feats["B3_rt_mean"]  = seq_mean(df["B3-2"])
    feats["B3_rt_std"]   = seq_std(df["B3-2"])

    print("Step 5: B4 feature 생성...")
    feats["B4_acc_rate"] = seq_rate(df["B4-1"], "1")
    feats["B4_rt_mean"]  = seq_mean(df["B4-2"])
    feats["B4_rt_std"]   = seq_std(df["B4-2"])

    print("Step 6: B5 feature 생성...")
    feats["B5_acc_rate"] = seq_rate(df["B5-1"], "1")
    feats["B5_rt_mean"]  = seq_mean(df["B5-2"])
    feats["B5_rt_std"]   = seq_std(df["B5-2"])

    print("Step 7: B6~B8 feature 생성...")
    feats["B6_acc_rate"] = seq_rate(df["B6"], "1")
    feats["B7_acc_rate"] = seq_rate(df["B7"], "1")
    feats["B8_acc_rate"] = seq_rate(df["B8"], "1")

    print("Step 8: 시퀀스 컬럼 drop & concat...")
    seq_cols = [
        "B1-1","B1-2","B1-3",
        "B2-1","B2-2","B2-3",
        "B3-1","B3-2",
        "B4-1","B4-2",
        "B5-1","B5-2",
        "B6","B7","B8"
    ]
    print("B 검사 데이터 전처리 완료")
    out = pd.concat([df.drop(columns=seq_cols, errors="ignore"), feats], axis=1)
    out.replace([np.inf,-np.inf], np.nan, inplace=True)
    return out

In [ ]:
# 학습 때 사용한 파생 (그대로)
def _has(df, cols):  return all(c in df.columns for c in cols)
def _safe_div(a, b, eps=1e-6): return a / (b + eps)

def add_features_A(df: pd.DataFrame) -> pd.DataFrame:
    feats = df.copy(); eps = 1e-6
    if _has(feats, ["Year","Month"]):
        feats["YearMonthIndex"] = feats["Year"] * 12 + feats["Month"]

    if _has(feats, ["A1_rt_mean","A1_resp_rate"]):
        feats["A1_speed_acc_tradeoff"] = _safe_div(feats["A1_rt_mean"], feats["A1_resp_rate"], eps)
    if _has(feats, ["A2_rt_mean","A2_resp_rate"]):
        feats["A2_speed_acc_tradeoff"] = _safe_div(feats["A2_rt_mean"], feats["A2_resp_rate"], eps)
    if _has(feats, ["A4_rt_mean","A4_acc_rate"]):
        feats["A4_speed_acc_tradeoff"] = _safe_div(feats["A4_rt_mean"], feats["A4_acc_rate"], eps)

    for k in ["A1","A2","A3","A4"]:
        m, s = f"{k}_rt_mean", f"{k}_rt_std"
        if _has(feats, [m, s]):
            feats[f"{k}_rt_cv"] = _safe_div(feats[s], feats[m], eps)

    for name, base in [
        ("A1_rt_side_gap_abs",  "A1_rt_side_diff"),
        ("A1_rt_speed_gap_abs", "A1_rt_speed_diff"),
        ("A2_rt_cond1_gap_abs", "A2_rt_cond1_diff"),
        ("A2_rt_cond2_gap_abs", "A2_rt_cond2_diff"),
        ("A4_stroop_gap_abs",   "A4_stroop_diff"),
        ("A4_color_gap_abs",    "A4_rt_color_diff"),
    ]:
        if base in feats.columns:
            feats[name] = feats[base].abs()

    if _has(feats, ["A3_valid_ratio","A3_invalid_ratio"]):
        feats["A3_valid_invalid_gap"] = feats["A3_valid_ratio"] - feats["A3_invalid_ratio"]
    if _has(feats, ["A3_correct_ratio","A3_invalid_ratio"]):
        feats["A3_correct_invalid_gap"] = feats["A3_correct_ratio"] - feats["A3_invalid_ratio"]
    if _has(feats, ["A5_acc_change","A5_acc_nonchange"]):
        feats["A5_change_nonchange_gap"] = feats["A5_acc_change"] - feats["A5_acc_nonchange"]

    feats.replace([np.inf, -np.inf], np.nan, inplace=True)
    return feats

def add_features_B(df: pd.DataFrame) -> pd.DataFrame:
    feats = df.copy(); eps = 1e-6
    if _has(feats, ["Year","Month"]):
        feats["YearMonthIndex"] = feats["Year"] * 12 + feats["Month"]

    for k, acc_col, rt_col in [
        ("B1", "B1_acc_task1", "B1_rt_mean"),
        ("B2", "B2_acc_task1", "B2_rt_mean"),
        ("B3", "B3_acc_rate",  "B3_rt_mean"),
        ("B4", "B4_acc_rate",  "B4_rt_mean"),
        ("B5", "B5_acc_rate",  "B5_rt_mean"),
    ]:
        if _has(feats, [rt_col, acc_col]):
            feats[f"{k}_speed_acc_tradeoff"] = _safe_div(feats[rt_col], feats[acc_col], eps)

    for k in ["B1","B2","B3","B4","B5"]:
        m, s = f"{k}_rt_mean", f"{k}_rt_std"
        if _has(feats, [m, s]):
            feats[f"{k}_rt_cv"] = _safe_div(feats[s], feats[m], eps)

    parts = []
    for k in ["B4","B5"]:
        if _has(feats, [f"{k}_rt_cv"]):
            parts.append(0.25 * feats[f"{k}_rt_cv"].fillna(0))
    for k in ["B3","B4","B5"]:
        acc = f"{k}_acc_rate" if k not in ["B1","B2"] else None
        if k in ["B1","B2"]:
            acc = f"{k}_acc_task1"
        if acc in feats:
            parts.append(0.25 * (1 - feats[acc].fillna(0)))
    for k in ["B1","B2"]:
        tcol = f"{k}_speed_acc_tradeoff"
        if tcol in feats:
            parts.append(0.25 * feats[tcol].fillna(0))
    if parts:
        feats["RiskScore_B"] = sum(parts)

    feats.replace([np.inf, -np.inf], np.nan, inplace=True)
    return feats

In [ ]:
# 정렬/보정 (모델이 학습 때 본 피처 순서로)
DROP_COLS = ["Test_id","Test","PrimaryKey","Age","TestDate"]

def align_to_model(X_df, model):
    feat_names = list(getattr(model, "feature_name_", []))
    if not feat_names:
        # fallback: 그냥 숫자형만
        X = X_df.select_dtypes(include=[np.number]).copy()
        return X.fillna(0.0)
    X = X_df.drop(columns=[c for c in DROP_COLS if c in X_df.columns], errors="ignore").copy()
    # 누락 피처 0으로 채움
    for c in feat_names:
        if c not in X.columns:
            X[c] = 0.0
    # 초과 피처 드롭 + 순서 일치
    X = X[feat_names]
    return X.apply(pd.to_numeric, errors="coerce").fillna(0.0)


In [ ]:
# main
def main():
    # ---- 경로 변수 (필요에 따라 수정) ----
    TEST_DIR  = "./data"              # test.csv, A.csv, B.csv, sample_submission.csv 위치
    MODEL_DIR = "./model"             # lgbm_A.pkl, lgbm_B.pkl 위치
    OUT_DIR   = "./output"
    SAMPLE_SUB_PATH = os.path.join(TEST_DIR, "/content/drive/MyDrive/data/sample_submission.csv")
    OUT_PATH  = os.path.join(OUT_DIR, "submission.csv")

    # ---- 모델 로드 ----
    print("Load models...")
    model_A = joblib.load(os.path.join(MODEL_DIR, "lgbm_A.pkl"))
    model_B = joblib.load(os.path.join(MODEL_DIR, "lgbm_B.pkl"))
    print(" OK.")

    # ---- 테스트 데이터 로드 ----
    print("Load test data...")
    meta = pd.read_csv(os.path.join(TEST_DIR, "/content/drive/MyDrive/data/test.csv"))
    Araw = pd.read_csv(os.path.join(TEST_DIR, "/content/drive/MyDrive/data/test/A.csv"))
    Braw = pd.read_csv(os.path.join(TEST_DIR, "/content/drive/MyDrive/data/test/B.csv"))
    print(f" meta={len(meta)}, Araw={len(Araw)}, Braw={len(Braw)}")

    # ---- 매핑 ----
    A_df = meta.loc[meta["Test"] == "A", ["Test_id", "Test"]].merge(Araw, on="Test_id", how="left")
    B_df = meta.loc[meta["Test"] == "B", ["Test_id", "Test"]].merge(Braw, on="Test_id", how="left")
    print(f" mapped: A={len(A_df)}, B={len(B_df)}")

    # ---- 전처리 → 파생 (학습과 동일) ----
    A_feat = add_features_A(preprocess_A(A_df)) if len(A_df) else pd.DataFrame()
    B_feat = add_features_B(preprocess_B(B_df)) if len(B_df) else pd.DataFrame()

    # ---- 피처 정렬/보정 ----
    XA = align_to_model(A_feat, model_A) if len(A_feat) else pd.DataFrame(columns=getattr(model_A,"feature_name_",[]))
    XB = align_to_model(B_feat, model_B) if len(B_feat) else pd.DataFrame(columns=getattr(model_B,"feature_name_",[]))
    print(f" aligned: XA={XA.shape}, XB={XB.shape}")

    # ---- 예측 ----
    print("Inference Model...")
    predA = model_A.predict_proba(XA)[:,1] if len(XA) else np.array([])
    predB = model_B.predict_proba(XB)[:,1] if len(XB) else np.array([])

    # ---- Test_id와 합치기 ----
    subA = pd.DataFrame({"Test_id": A_df["Test_id"].values, "prob": predA})
    subB = pd.DataFrame({"Test_id": B_df["Test_id"].values, "prob": predB})
    probs = pd.concat([subA, subB], axis=0, ignore_index=True)

    # ---- sample_submission 기반 결과 생성 (Label 컬럼에 0~1 확률 채움) ----
    os.makedirs(OUT_DIR, exist_ok=True)
    sample = pd.read_csv(SAMPLE_SUB_PATH)
    # sample의 Test_id 순서에 맞추어 prob 병합
    out = sample.merge(probs, on="Test_id", how="left")
    out["Label"] = out["prob"].astype(float).fillna(0.0)
    out = out.drop(columns=["prob"])

    out.to_csv(OUT_PATH, index=False)
    print(f"✅ Saved: {OUT_PATH} (rows={len(out)})")

if __name__ == "__main__":
    main()

Load models...
 OK.
Load test data...
 meta=10, Araw=4, Braw=6
 mapped: A=4, B=6
Step 1: Age, TestDate 파생...
Step 2: A1 feature 생성...


100%|██████████| 4/4 [00:00<00:00, 3012.07it/s]


Step 3: A2 feature 생성...


100%|██████████| 4/4 [00:00<00:00, 5424.25it/s]


Step 4: A3 feature 생성...


...

100%|██████████| 6/6 [00:00<00:00, 6300.91it/s]


Step 7: B6~B8 feature 생성...


100%|██████████| 6/6 [00:00<00:00, 7605.27it/s]

Step 8: 시퀀스 컬럼 drop & concat...
B 검사 데이터 전처리 완료


 aligned: XA=(4, 62), XB=(6, 46)
Inference Model...
✅ Saved: ./output/submission.csv (rows=10)


#제출파일 생성

In [42]:
# submit_dir 변수가 없어서 다시 정의
from pathlib import Path

submit_dir = Path("/content/drive/MyDrive/data/submit_package")
submit_dir.mkdir(parents=True, exist_ok=True)

# script.py 다시 저장
script_path = submit_dir / "script.py"
script_path.write_text(script_py.strip(), encoding="utf-8")

# 결과 확인
script_path.name


'script.py'

In [43]:
# 1단계: script.py 생성 (inference.py와 동일 내용)

script_py = """
import os
import pandas as pd
import numpy as np
import joblib

from preprocess import preprocess_A, preprocess_B
from features import add_features_A, add_features_B
from align import align_to_model

def inference(test_dir: str = "./data", model_dir: str = "./model", output_dir: str = "./output"):
    sample_sub_path = os.path.join(test_dir, "sample_submission.csv")
    out_path = os.path.join(output_dir, "submission.csv")

    print("Loading models...")
    model_A = joblib.load(os.path.join(model_dir, "lgbm_A.pkl"))
    model_B = joblib.load(os.path.join(model_dir, "lgbm_B.pkl"))
    print("Models loaded.")

    print("Loading test data...")
    meta = pd.read_csv(os.path.join(test_dir, "test.csv"))
    Araw = pd.read_csv(os.path.join(test_dir, "A.csv"))
    Braw = pd.read_csv(os.path.join(test_dir, "B.csv"))

    A_df = meta.loc[meta["Test"] == "A", ["Test_id", "Test"]].merge(Araw, on="Test_id", how="left")
    B_df = meta.loc[meta["Test"] == "B", ["Test_id", "Test"]].merge(Braw, on="Test_id", how="left")

    A_feat = add_features_A(preprocess_A(A_df)) if len(A_df) else pd.DataFrame()
    B_feat = add_features_B(preprocess_B(B_df)) if len(B_df) else pd.DataFrame()

    XA = align_to_model(A_feat, model_A) if len(A_feat) else pd.DataFrame(columns=getattr(model_A,"feature_name_",[]))
    XB = align_to_model(B_feat, model_B) if len(B_feat) else pd.DataFrame(columns=getattr(model_B,"feature_name_",[]))

    print("Running inference...")
    predA = model_A.predict_proba(XA)[:,1] if len(XA) else np.array([])
    predB = model_B.predict_proba(XB)[:,1] if len(XB) else np.array([])

    subA = pd.DataFrame({"Test_id": A_df["Test_id"].values, "prob": predA})
    subB = pd.DataFrame({"Test_id": B_df["Test_id"].values, "prob": predB})
    probs = pd.concat([subA, subB], axis=0, ignore_index=True)

    os.makedirs(output_dir, exist_ok=True)
    sample = pd.read_csv(sample_sub_path)
    out = sample.merge(probs, on="Test_id", how="left")
    out["Label"] = out["prob"].astype(float).fillna(0.0)
    out.drop(columns=["prob"], inplace=True)
    out.to_csv(out_path, index=False)
    print(f"✅ Inference completed. Saved to: {out_path}")

if __name__ == "__main__":
    inference()
"""

# 저장
script_path = submit_dir / "script.py"
script_path.write_text(script_py.strip(), encoding="utf-8")

# 결과 확인
script_path.name


'script.py'

In [44]:
# 2단계: preprocess.py 생성

preprocess_py = """
import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

def convert_age(val):
    if pd.isna(val): return np.nan
    try:
        base = int(str(val)[:-1])
        return base if str(val)[-1] == "a" else base + 5
    except:
        return np.nan

def split_testdate(val):
    try:
        v = int(val)
        return v // 100, v % 100
    except:
        return np.nan, np.nan

def seq_mean(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").mean() if x else np.nan
    )

def seq_std(series):
    return series.fillna("").progress_apply(
        lambda x: np.fromstring(x, sep=",").std() if x else np.nan
    )

def seq_rate(series, target="1"):
    return series.fillna("").progress_apply(
        lambda x: str(x).split(",").count(target) / len(x.split(",")) if x else np.nan
    )

def masked_mean_from_csv_series(cond_series, val_series, mask_val):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)
    mask = (cond_arr == mask_val)
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts==0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)

def masked_mean_in_set_series(cond_series, val_series, mask_set):
    cond_df = cond_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    val_df  = val_series.fillna("").str.split(",", expand=True).replace("", np.nan)
    cond_arr = cond_df.to_numpy(dtype=float)
    val_arr  = val_df.to_numpy(dtype=float)
    mask = np.isin(cond_arr, list(mask_set))
    with np.errstate(invalid="ignore"):
        sums = np.nansum(np.where(mask, val_arr, np.nan), axis=1)
        counts = np.sum(mask, axis=1)
        out = sums / np.where(counts == 0, np.nan, counts)
    return pd.Series(out, index=cond_series.index)
"""

# 저장
preprocess_path = submit_dir / "preprocess.py"
preprocess_path.write_text(preprocess_py.strip(), encoding="utf-8")

# 완료 후 다음 단계로 안내
preprocess_path.name


'preprocess.py'

In [45]:
# 3단계: features.py 생성

features_py = """
import numpy as np
import pandas as pd
from preprocess import *

def preprocess_A(df):
    df = df.copy()
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)
    feats["A1_resp_rate"] = seq_rate(df["A1-3"], "1")
    feats["A1_rt_mean"]   = seq_mean(df["A1-4"])
    feats["A1_rt_std"]    = seq_std(df["A1-4"])
    feats["A1_rt_left"]   = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 1)
    feats["A1_rt_right"]  = masked_mean_from_csv_series(df["A1-1"], df["A1-4"], 2)
    feats["A1_rt_side_diff"] = feats["A1_rt_left"] - feats["A1_rt_right"]
    feats["A1_rt_slow"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 1)
    feats["A1_rt_fast"]   = masked_mean_from_csv_series(df["A1-2"], df["A1-4"], 3)
    feats["A1_rt_speed_diff"] = feats["A1_rt_slow"] - feats["A1_rt_fast"]
    df.drop(columns=[col for col in df.columns if col.startswith("A1")], inplace=True)
    return pd.concat([df, feats], axis=1)

def preprocess_B(df):
    df = df.copy()
    df["Age_num"] = df["Age"].map(convert_age)
    ym = df["TestDate"].map(split_testdate)
    df["Year"] = [y for y, m in ym]
    df["Month"] = [m for y, m in ym]

    feats = pd.DataFrame(index=df.index)
    feats["B1_acc_task1"] = seq_rate(df["B1-1"], "1")
    feats["B1_rt_mean"]   = seq_mean(df["B1-2"])
    feats["B1_rt_std"]    = seq_std(df["B1-2"])
    feats["B1_acc_task2"] = seq_rate(df["B1-3"], "1")
    df.drop(columns=[col for col in df.columns if col.startswith("B1")], inplace=True)
    return pd.concat([df, feats], axis=1)

def _has(df, cols): return all(c in df.columns for c in cols)
def _safe_div(a, b, eps=1e-6): return a / (b + eps)

def add_features_A(df):
    feats = df.copy()
    if _has(feats, ["A1_rt_mean","A1_resp_rate"]):
        feats["A1_speed_acc_tradeoff"] = _safe_div(feats["A1_rt_mean"], feats["A1_resp_rate"])
    return feats

def add_features_B(df):
    feats = df.copy()
    if _has(feats, ["B1_rt_mean","B1_acc_task1"]):
        feats["B1_speed_acc_tradeoff"] = _safe_div(feats["B1_rt_mean"], feats["B1_acc_task1"])
    return feats
"""

# 저장
features_path = submit_dir / "features.py"
features_path.write_text(features_py.strip(), encoding="utf-8")

# 완료 후 다음 단계 안내
features_path.name


'features.py'

In [46]:
# 4단계: align.py 생성

align_py = """
import pandas as pd

DROP_COLS = ["Test_id", "Test", "PrimaryKey", "Age", "TestDate"]

def align_to_model(X_df, model):
    feat_names = list(getattr(model, "feature_name_", []))
    if not feat_names:
        X = X_df.select_dtypes(include=[float, int])
        return X.fillna(0.0)

    X = X_df.drop(columns=[c for c in DROP_COLS if c in X_df.columns], errors="ignore").copy()
    for c in feat_names:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feat_names]
    return X.apply(pd.to_numeric, errors="coerce").fillna(0.0)
"""

# 저장
align_path = submit_dir / "align.py"
align_path.write_text(align_py.strip(), encoding="utf-8")

# 다음 단계로
align_path.name


'align.py'

In [47]:
# 5단계: requirements.txt 생성

requirements_txt = """
pandas
numpy
lightgbm
joblib
tqdm
"""

# 저장
requirements_path = submit_dir / "requirements.txt"
requirements_path.write_text(requirements_txt.strip(), encoding="utf-8")

# 다음 단계로
requirements_path.name


'requirements.txt'

In [64]:
# README.md 내용
readme_md = """
# 교통사고 예측 AI - 코드 제출 가이드

이 코드는 교통사고 예측 AI 경진대회를 위한 추론 스크립트입니다.

## 📁 디렉토리 구조
submit_package/
├── script.py             # 메인 추론 스크립트 (필수)
├── preprocess.py         # 공통 전처리 함수
├── features.py           # A/B 전처리 및 피처 생성
├── align.py              # 모델 입력 피처 정렬
├── requirements.txt      # 필요 패키지 목록
├── README.md             # 실행 방법 설명
├── model/
│   ├── lgbm_A.pkl
│   └── lgbm_B.pkl
└── data/
    ├── test.csv
    ├── A.csv
    ├── B.csv
    └── sample_submission.csv

## 🚀 실행 방법

python script.py

- 평가 시스템에서는 기본 경로(./data, ./model, ./output)를 기준으로 실행됩니다.

## 💡 주의 사항

- 인터넷 없이 오프라인 환경에서 실행됩니다.
- 전체 실행 시간은 30분 이내여야 합니다.
- 모델 및 데이터 파일은 반드시 해당 폴더에 위치시켜야 합니다.

---
감사합니다!
"""

# 파일 저장
readme_path = submit_dir / "README.md"
readme_path.write_text(readme_md.strip(), encoding="utf-8")

readme_path.name


'README.md'

In [50]:
# 7단계: model/ 및 data/ 폴더 생성

model_dir = submit_dir / "model"
data_dir = submit_dir / "data"

model_dir.mkdir(exist_ok=True)
data_dir.mkdir(exist_ok=True)

# 생성된 폴더 목록 확인
folder_list = sorted([f.name for f in submit_dir.iterdir() if f.is_dir()])
folder_list


['data', 'model']

In [65]:
# script.py 수정: 올바른 import 경로로 변경
fixed_script_py = """
import os
import pandas as pd
import numpy as np
import joblib

from features import preprocess_A, preprocess_B
from features import add_features_A, add_features_B
from align import align_to_model

def inference(test_dir: str = "./data", model_dir: str = "./model", output_dir: str = "./output"):
    sample_sub_path = os.path.join(test_dir, "sample_submission.csv")
    out_path = os.path.join(output_dir, "submission.csv")

    print("Loading models...")
    model_A = joblib.load(os.path.join(model_dir, "lgbm_A.pkl"))
    model_B = joblib.load(os.path.join(model_dir, "lgbm_B.pkl"))
    print("Models loaded.")

    print("Loading test data...")
    meta = pd.read_csv(os.path.join(test_dir, "test.csv"))
    Araw = pd.read_csv(os.path.join(test_dir, "A.csv"))
    Braw = pd.read_csv(os.path.join(test_dir, "B.csv"))

    A_df = meta.loc[meta["Test"] == "A", ["Test_id", "Test"]].merge(Araw, on="Test_id", how="left")
    B_df = meta.loc[meta["Test"] == "B", ["Test_id", "Test"]].merge(Braw, on="Test_id", how="left")

    A_feat = add_features_A(preprocess_A(A_df)) if len(A_df) else pd.DataFrame()
    B_feat = add_features_B(preprocess_B(B_df)) if len(B_df) else pd.DataFrame()

    XA = align_to_model(A_feat, model_A) if len(A_feat) else pd.DataFrame(columns=getattr(model_A,"feature_name_",[]))
    XB = align_to_model(B_feat, model_B) if len(B_feat) else pd.DataFrame(columns=getattr(model_B,"feature_name_",[]))

    print("Running inference...")
    predA = model_A.predict_proba(XA)[:,1] if len(XA) else np.array([])
    predB = model_B.predict_proba(XB)[:,1] if len(XB) else np.array([])

    subA = pd.DataFrame({"Test_id": A_df["Test_id"].values, "prob": predA})
    subB = pd.DataFrame({"Test_id": B_df["Test_id"].values, "prob": predB})
    probs = pd.concat([subA, subB], axis=0, ignore_index=True)

    os.makedirs(output_dir, exist_ok=True)
    sample = pd.read_csv(sample_sub_path)
    out = sample.merge(probs, on="Test_id", how="left")
    out["Label"] = out["prob"].astype(float).fillna(0.0)
    out.drop(columns=["prob"], inplace=True)
    out.to_csv(out_path, index=False)
    print(f"✅ Inference completed. Saved to: {out_path}")

if __name__ == "__main__":
    inference()
"""

# 덮어쓰기 저장
script_path = submit_dir / "script.py"
script_path.write_text(fixed_script_py.strip(), encoding="utf-8")

# 완료 알림
script_path.name


'script.py'